## Trading a Delta Hedged Straddle

1.  Buy 10 contracts of puts and calls at the strike closest to current stock price every morning
    a. Expiry that is closest to today but at least 5 trading days out.
    b. Hedge remaining delta
2.  Re-hedge if residual delta > 2 contracts
2.  Exit the trade at EOD.
5.  We assume no slippage (i.e. entry and exit at mid price) and commission of 1/2 a cent for stock trades and 1 dollar for option trades

First lets generate some option prices based on the underlying stock price.  We will generate some random volatility numbers, and use Black Scholes to generate prices.  We will add all this data to a strategy context that we can later use from our strategy and avoid excessive global variables.

In [1]:
%%checkall
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Sequence
import pyqstrat as pq
from types import SimpleNamespace


_logger = pq.get_child_logger(__name__)


def get_symbol(put_call: str, strike: int, expiry: np.datetime64) -> str:
    '''Create option symbol from parameters'''
    return f'{put_call}-{strike}-{expiry}'


def parse_symbol(symbol: str) -> tuple[str, int, np.datetime64]:
    '''Break down option symbol into put_call, strike, expiry'''
    split = symbol.split('-', maxsplit=2)
    return (split[0], int(split[1]), np.datetime64(split[2]))
       

@dataclass
class StraddleEntryRule:
    expiries: dict[np.datetime64, np.ndarray]
    strikes: dict[tuple[np.datetime64, str, np.datetime64], np.ndarray]
    umids: dict[np.datetime64, float]
        
    def __call__(self,
                 contract_group: pq.ContractGroup,
                 i: int,
                 timestamps: np.ndarray,
                 indicator_values: pq.SimpleNamespace,
                 signal_values: np.ndarray,
                 account: pq.Account,
                 current_orders: Sequence[pq.Order],
                 strategy_context: pq.StrategyContextType) -> list[pq.Order]:
        timestamp = timestamps[i]
        date = timestamp.astype('M8[D]')
        expiries = self.expiries[date]
        expiry = expiries[pq.np_find_closest(expiries, date + np.timedelta64(30, 'D'))]
        put_strikes = self.strikes[(date, 'P', expiry)]
        call_strikes = self.strikes[(date, 'C', expiry)]
        umid = self.umids[timestamp]
        cidx = pq.np_find_closest(call_strikes, umid)
        pidx = pq.np_find_closest(put_strikes, umid)
        found = False
        for j in range(10):
            put_strike = put_strikes[pidx]
            call_strike = call_strikes[cidx]
            put_symbol = get_symbol('P', put_strike, expiry)
            call_symbol = get_symbol('C', call_strike, expiry)
            if pq.Contract.exists(put_symbol):
                put_contract = pq.Contract.get(put_symbol)
            else:
                put_contract = pq.Contract.create(put_symbol, contract_group, expiry, 100)
            if pq.Contract.exists(call_symbol):
                call_contract = pq.Contract.get(call_symbol)
            else:
                call_contract = pq.Contract.create(call_symbol, contract_group, expiry, 100)
            put_delta = context.get_delta(put_contract, timestamps, i, context)
            call_delta = context.get_delta(call_contract, timestamps, i, context)
            if np.isfinite(put_delta) and np.isfinite(call_delta): 
                found = True
                break
            cidx += 1
            pidx -= 1
        if not found: return []
        symbol = f'{put_symbol}_{call_symbol}'
        if pq.Contract.exists(symbol):
            contract = pq.Contract.get(symbol)
        else:
            assert put_contract is not None
            assert call_contract is not None
            contract = pq.Contract.create(symbol, 
                                          contract_group,
                                          multiplier=100, 
                                          components=[(put_contract, 1.), (call_contract, 1.)])
        assert contract is not None
        order = pq.MarketOrder(contract=contract, timestamp=timestamp, qty=10, reason_code='ENTER_STRADDLE')
        _logger.info(f'ORDER: {timestamp} {order}')
        return [order]
    
    
@dataclass
class RehedgeRule:
    '''
    A rule to re-hedge deltas on a straddle
    Args:
        reason_code: the reason for the orders (used for display purposes)
        price_func: the function this rule uses to get market prices
    '''
    reason_code: str
    price_func: pq.PriceFunctionType
        
    def __init__(self, 
                 reason_code: str, 
                 price_func: pq.PriceFunctionType) -> None:
        self.reason_code = reason_code
        self.price_func = price_func
        
    def __call__(self,
                 contract_group: pq.ContractGroup,
                 i: int,
                 timestamps: np.ndarray,
                 indicator_values: SimpleNamespace,
                 signal_values: np.ndarray,
                 account: pq.Account,
                 current_orders: Sequence[pq.Order],
                 strategy_context: pq.StrategyContextType) -> list[pq.Order]:
        timestamp = timestamps[i]
        positions = account.positions(pq.ContractGroup.get('OPTIONS'), timestamp)
        
        orders: list[pq.Order] = []
        for (contract, qty) in positions:
            if not contract.is_basket(): continue
            put_contract, put_ratio = contract.components[0]
            call_contract, call_ratio = contract.components[1]
            put_qty = int(round(qty * call_ratio))
            call_qty = int(round(qty * put_ratio))
            hedge_contract, target_hedge_qty = get_hedge(
                put_contract, call_contract, put_qty, call_qty, timestamps, i, context)
            if np.isnan(target_hedge_qty): return []  # can have nan deltas sometimes
            hedge_positions = account.positions(contract_group, timestamp)
            pq.assert_(len(hedge_positions) in [0, 1], f'unexpected num of hedge positions: {hedge_positions}')
            curr_hedge_qty = 0
            if len(hedge_positions) == 1:
                curr_hedge_qty = int(round(hedge_positions[0][1]))
            hedge_qty = target_hedge_qty - curr_hedge_qty
            if hedge_qty == 0: return []
            hedge_order = pq.MarketOrder(contract=hedge_contract, 
                                         timestamp=timestamp, 
                                         qty=int(hedge_qty), 
                                         reason_code='REHEDGE')
            orders.append(hedge_order)
            _logger.info(f'ORDER: {timestamp} {hedge_order}')
        return orders
    

def get_hedge(put: pq.Contract, 
              call: pq.Contract, 
              put_qty: int, 
              call_qty: int, 
              timestamps: np.ndarray, 
              i: int, 
              context: pq.StrategyContextType) -> tuple[pq.Contract, int]:
    delta: float = 0
    delta += context.get_delta(put, timestamps, i, context) * put_qty
    delta += context.get_delta(call, timestamps, i, context) * call_qty
    if np.isnan(delta): delta = 0
    hedge_contract = pq.Contract.get('SPX')
    assert hedge_contract is not None
    hedge_qty = int(np.round(-100 * delta))
    return hedge_contract, hedge_qty


def get_expiries(prices: pd.DataFrame) -> dict[np.datetime64, np.ndarray]:
    _expiries = prices.groupby(['date']).expiry.unique()
    expiries = {_expiries.index.values[i].astype('M8[D]'): 
                np.sort(_expiries.values[i].astype('M8[D]')) for i in range(len(_expiries))}
    return expiries


def get_strikes(prices: pd.DataFrame) -> dict[tuple[np.datetime64, str, np.datetime64], np.ndarray]:
    _strikes = prices.groupby(['date', 'put_call', 'expiry']).strike.unique()
    for i in range(len(_strikes)):
        _strikes[i] = _strikes[i][_strikes[i] % 100 == 0]
    strikes = {(_strikes.index.get_level_values(0).values[i].astype('M8[D]'), 
                _strikes.index.get_level_values(1)[i], 
                _strikes.index.get_level_values(2).values[i].astype('M8[D]')): np.sort(_strikes.values[i]) 
               for i in range(len(_strikes))}
    return strikes


def get_price_function(prices: pd.DataFrame, field_name: str) -> pq.PriceFunctionType:
    price_dict: dict[str, tuple[np.ndarray, np.ndarray]] = {}
    for symbol in np.unique(prices.symbol.values):
        sym_prc = prices[['timestamp', field_name]][prices.symbol == symbol].sort_values(by='timestamp')
        _timestamps = sym_prc.timestamp.values.astype('M8[m]')
        _prices = sym_prc[field_name].values
        price_dict[symbol] = (_timestamps, _prices)
    spx_prices = prices[['timestamp', 'umid']].sort_values(by=['timestamp']).drop_duplicates(subset=['timestamp'])
    price_dict['SPX'] = (spx_prices.timestamp.values.astype('M8[m]'), spx_prices.umid.values)
    return pq.PriceFuncArrayDict(price_dict=price_dict)


if __name__ == '__main__':
    pq.set_defaults()

    prices = pd.read_csv('./data/spx_options.csv.gz', parse_dates=['timestamp'])
    prices = prices[['timestamp', 'symbol', 'umid', 'c', 'delta']]
    
    prices['date'] = prices.timestamp.values.astype('M8[D]')
    # prices = prices[prices.date == "2023-01-03"]

    # remove prices outside regular trading hours (9:30 am and 4 pm)
    minute = (prices.timestamp - prices.date) / np.timedelta64(1, 'm')
    prices = prices[(minute > 9 * 60 + 30) & (minute < 16 * 60)]

    splits = prices.symbol.str.split('-', n=2, expand=True)
    prices['put_call'] = splits[0]
    prices['strike'] = splits[1].astype(int)
    prices['expiry'] = splits[2].astype(np.datetime64)

    data = prices[['timestamp', 'umid']].sort_values(by=['timestamp']).drop_duplicates(subset=['timestamp'])
    data['date'] = data.timestamp.values.astype('M8[D]')
    data['hour'] = data.timestamp.dt.hour
    
    # Beginning and end of day and rehedge signals
    # Try 6 five-minute periods in case we don't get valid prices (every strike is not traded in every bar)
    bod = pd.Series(np.where(data.date != data.date.shift(1), 1, np.nan))
    bod = bod.fillna(method='ffill', limit=6)  
    data['bod'] = np.where(bod == 1, True, False)
    
    eod = pd.Series(np.where(data.date != data.date.shift(-1), 1, np.nan))
    # Try getting out 30 minutes before close so we have 6 bars to try and get out
    eod = eod.fillna(method='bfill', limit=6)  
    data['eod'] = np.where(eod == 1, True, False)
    
    data['rehedge'] = (data.hour != data.hour.shift(1))
    
    strat_builder = pq.StrategyBuilder(data)
    context = pq.StrategyContextType()
    price_func = get_price_function(prices, 'c')
    delta_func = get_price_function(prices, 'delta')
    context.get_price = price_func
    context.get_delta = delta_func
    strat_builder.set_strategy_context(context)
    strat_builder.set_price_function(price_func)
    
    expiries = get_expiries(prices)
    strikes = get_strikes(prices)
    umids = {data.timestamp.values[i].astype('M8[m]'): data.umid.values[i] for i in range(len(data))}

    opt_cg = pq.ContractGroup.get('OPTIONS')
    strat_builder.add_contract_group(opt_cg)

    hedge_cg = pq.ContractGroup.get('HEDGES')
    spx = pq.Contract.create('SPX', contract_group=hedge_cg)
    strat_builder.add_contract_group(hedge_cg)

    # add rules to the strategy
    straddle_entry_rule = StraddleEntryRule(expiries, strikes, umids)
    strat_builder.add_series_rule('bod', straddle_entry_rule, position_filter='zero', contract_groups=[opt_cg])
    rehedge_rule = RehedgeRule('REHEDGE', context.get_price)
    strat_builder.add_series_rule('rehedge', rehedge_rule, contract_groups=[hedge_cg])
    close_rule = pq.ClosePositionExitRule('EOD', context.get_price)
    strat_builder.add_series_rule('eod', close_rule, position_filter='nonzero')

    # build and run strategy
    strategy = strat_builder()
    strategy.run()

running typecheck
Success: no issues found in 1 source file

running flake8
flake8 success
[2023-10-23 20:27:36.767 __call__] ORDER: 2023-01-03T09:35 P-3900-2023-01-20_C-3900-2023-01-20 2023-01-03 09:35:00 qty: 10 ENTER_STRADDLE  OrderStatus.OPEN
[2023-10-23 20:27:36.767 __call__] TRADE: 2023-01-03T09:40 P-3900-2023-01-20_C-3900-2023-01-20  2023-01-03 09:40:00 qty: 10 prc: 143.65   order: P-3900-2023-01-20_C-3900-2023-01-20 2023-01-03 09:35:00 qty: 10 ENTER_STRADDLE  OrderStatus.OPEN 
[2023-10-23 20:27:36.768 __call__] ORDER: 2023-01-03T10:00 SPX 2023-01-03 10:00:00 qty: 228 REHEDGE  OrderStatus.OPEN
[2023-10-23 20:27:36.768 __call__] TRADE: 2023-01-03T10:05 SPX  2023-01-03 10:05:00 qty: 228 prc: 3835.12   order: SPX 2023-01-03 10:00:00 qty: 228 REHEDGE  OrderStatus.OPEN 
[2023-10-23 20:27:36.768 __call__] ORDER: 2023-01-03T11:00 SPX 2023-01-03 11:00:00 qty: -228 REHEDGE  OrderStatus.OPEN
[2023-10-23 20:27:36.769 __call__] TRADE: 2023-01-03T11:05 SPX  2023-01-03 11:05:00 qty: -228 prc:

[2023-10-23 20:27:36.786 __call__] ORDER: 2023-01-06T13:00 SPX 2023-01-06 13:00:00 qty: 194 REHEDGE  OrderStatus.OPEN
[2023-10-23 20:27:36.786 __call__] TRADE: 2023-01-06T13:05 SPX  2023-01-06 13:05:00 qty: 194 prc: 3871   order: SPX 2023-01-06 13:00:00 qty: 194 REHEDGE  OrderStatus.OPEN 
[2023-10-23 20:27:36.786 __call__] ORDER: 2023-01-06T15:25 P-3600-2023-02-17_C-4000-2023-02-17 2023-01-06 15:25:00 qty: -10 EOD  OrderStatus.OPEN
[2023-10-23 20:27:36.787 __call__] TRADE: 2023-01-06T15:30 P-3600-2023-02-17_C-4000-2023-02-17  2023-01-06 15:30:00 qty: -10 prc: 87.54   order: P-3600-2023-02-17_C-4000-2023-02-17 2023-01-06 15:25:00 qty: -10 EOD  OrderStatus.OPEN 
[2023-10-23 20:27:36.788 __call__] ORDER: 2023-01-09T09:35 P-3900-2023-02-17_C-3900-2023-02-17 2023-01-09 09:35:00 qty: 10 ENTER_STRADDLE  OrderStatus.OPEN
[2023-10-23 20:27:36.788 __call__] ORDER: 2023-01-09T09:40 P-3800-2023-02-17_C-4000-2023-02-17 2023-01-09 09:40:00 qty: 10 ENTER_STRADDLE  OrderStatus.OPEN
[2023-10-23 20:27:3

In [8]:
strategy.df_roundtrip_trades()

,symbol,multiplier,entry_timestamp,exit_timestamp,qty,entry_price,exit_price,entry_reason,exit_reason,entry_commission,exit_commission,net_pnl
0,P-3900-2023-01-20_C-3900-2023-01-20,100,2023-01-03 09:40:00,2023-01-03 15:35:00,10,143.65,147.8,ENTER_STRADDLE,EOD,0,0,4150
1,SPX,1,2023-01-03 10:05:00,2023-01-03 11:05:00,228,3835.11951,3830.80603,REHEDGE,REHEDGE,0,0,-983.472665
2,SPX,1,2023-01-03 14:05:00,2023-01-03 15:05:00,343,3810.88904,3814.55151,REHEDGE,REHEDGE,0,0,1256.22911
3,P-3500-2023-02-17_C-4100-2023-02-17,100,2023-01-04 10:05:00,2023-01-04 15:35:00,10,45.3,44.27,ENTER_STRADDLE,EOD,0,0,-1030
4,SPX,1,2023-01-04 15:05:00,2023-01-04 15:30:00,-78,3845.28845,3839.80847,REHEDGE,EOD,0,0,427.438479
5,P-3600-2023-02-17_C-4000-2023-02-17,100,2023-01-05 09:40:00,2023-01-05 15:55:00,10,81.01,77.7,ENTER_STRADDLE,EOD,0,0,-3310
6,SPX,1,2023-01-05 10:05:00,2023-01-05 11:05:00,-50,3831.40906,3809.60742,REHEDGE,REHEDGE,0,0,1090.08178
7,SPX,1,2023-01-05 14:05:00,2023-01-05 15:05:00,-67,3821.97205,3817.46692,REHEDGE,REHEDGE,0,0,301.843506
8,P-3600-2023-02-17_C-4000-2023-02-17,100,2023-01-06 09:45:00,2023-01-06 15:30:00,10,75.82,87.54,ENTER_STRADDLE,EOD,0,0,11720
9,SPX,1,2023-01-06 10:05:00,2023-01-06 13:05:00,-95,3841.67346,3870.99707,REHEDGE,REHEDGE,0,0,-2785.7428


In [11]:
strategy.evaluate_returns();

,gmean,amean,std,shrp,srt,k,calmar,mar,mdd_pct,mdd_dates,dd_3y_pct,dd_3y_timestamps,up_dwn,2023
,0.5853,0.3862,0.00395,6.159,29.81,8.664,153.9,153.9,0.00251,2023-01-03/2023-01-05,0.00251,2023-01-03/2023-01-05,2/3/0.4,0.5853


2023-01-03T00:00 2023-01-05T00:00


In [12]:
pq.Strategy.evaluate_returns(pq.ContractGroup.get('HEDGES'))

AttributeError: 'ContractGroup' object has no attribute 'df_returns'